In [12]:
!git push

Counting objects: 387, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (355/355), done.
Writing objects: 100% (387/387), 762.78 KiB | 10.90 MiB/s, done.
Total 387 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   https://github.com/basilwong/sagemaker-repo.git
To https://github.com/basilwong/awstest1.git
   d49dcea..c7c5d82  master -> master


### Add Dependencies

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3

from datetime import datetime
import zipfile
import os
import json 
import uuid

# Installing src dependency.
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('src')

!pip install pydub
import audio_util
import processing_util

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



For the Corresponding IAM Role, add the following policies:

* AmazonTranscribeFullAccess
* AWSMarketplaceManageSubscriptions
* AmazonPollyFullAccess
* AmazonSageMakerFullAccess

### Creating the Model

In [3]:
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

Note that if the initial audio file is longer than around 30 seconds, it is too large for the model. The split_mp3() method in  src.audio_util works around this by splitting an mp3 file into 30 second segments. 

This method requires ffmpeg as a dependency sice it uses pydub. Instead of installing it on the notebook, the code below was executed locally with ffmpeg installed. (```apt-get install ffmpeg``` for an Ubunutu machine, as I had trouble figuring out how to install it via yum).

But no worries the output of the split_mp3() method has already been added to this repository so no need to go execute it for demo purposes. 

In [74]:
# The following lines of code require 
audio_util.split_mp3("songs/drake-toosie_slide.mp3", "../source-separation-input/")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pydub/utils.py:193: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe': 'ffprobe'

In [4]:
batch_input_folder = "source-separation-input"


transform_input = sagemaker_session.upload_data(batch_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [5]:
bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-13 04:48:15 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2020-04-13 04:48:15 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2020-04-13 04:48:15 +0000] [10] [INFO] Using worker: gevent
[2020-04-13 04:48:15 +0000] [14] [INFO] Booting worker with pid: 14
[2020-04-13 04:48:15 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-13 04:48:15 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-13 04:48:15 +0000] [17] [INFO] Booting worker with pid: 17
Testing...
Testing...
2020-04-13 04:48:39.079016: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753319

['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
Num of variables64
Pre-trained model restored for song prediction
Num of variables64
Pre-trained model restored for song prediction
169.254.255.130 - - [13/Apr/2020:04:54:47 +0000] "POST /invocations HTTP/1.1" 200 19035684 "-" "Go-http-client/1.1"
Input pa

### Processing the Batch Output

In [142]:
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
audio_util.clear_folder('source-separation-output/batch-transform-output')
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [148]:
# Extracting files from zip files. 
audio_util.clear_folder('source-separation-output/extracted')
for file in os.listdir('source-separation-output/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('source-separation-output/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('source-separation-output/extracted/'+file.split('.')[0]+'/')

output-3.zip
output-6.zip
output-8.zip
output-9.zip
output-1.zip
output-4.zip
output-7.zip
output-2.zip
output-5.zip


In [1]:
# Separating the vocal files and the background sound files.
audio_util.clear_folder('source-separation-output/vocals')
audio_util.clear_folder('source-separation-output/background')
for i, folder in enumerate(sorted(os.listdir('source-separation-output/extracted/'))):
    for file in os.listdir('source-separation-output/extracted/' + folder + '/output'):
        new_file_name = str(i).zfill(5) + ".wav"
        if "vocals" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/vocals/vocals' + new_file_name)
        elif "accompaniment" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/background/background' + new_file_name)

NameError: name 'audio_util' is not defined

### Transcribe the Vocal Files

In [151]:
# Upload the Vocal files onto s3
local_vocals_folder = "source-separation-output/vocals/"
transcribe_input_prefix = "transcribe-input"

transcribe_input = sagemaker_session.upload_data(local_vocals_folder, key_prefix=transcribe_input_prefix)
print("Transcribe input uploaded to " + transcribe_input)

Transcribe input uploaded to s3://sagemaker-us-east-2-075178354542/transcribe-input


In [ ]:
# Start a transcription job for each file. Add the transcription to finsihed jobs once finished. 
transcribe = boto3.client('transcribe')
output_bucket_name = "transcribe-output"
audio_util.clear_folder('transcribe-output')
uri_prefix = "https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/"
finished_jobs = list()

for file in sorted(os.listdir(local_vocals_folder)):

    print("Transcribing: " + file)
    job_uri = uri_prefix + file
    transcribe.start_transcription_job(
        TranscriptionJobName=file,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US'
    )
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=file)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
    
    api_data = requests.get(url=status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = api_data.json()
    finished_jobs.append(data)
    dump_file_name = 'transcribe-output/transcription' + file.split(".")[0] + '.json'
    # Writing to json files for analysis purposes.
    with open(dump_file_name, 'w') as f:
        json.dump(data, f, indent=4)
    transcribe.delete_transcription_job(TranscriptionJobName=file)
    
finished_jobs.sort(key=lambda x : x['jobName'])

Transcribing: vocals00000.wav
Transcribing: vocals00001.wav


In [158]:
transcribe.delete_transcription_job(TranscriptionJobName="vocals00000.wav")

{'ResponseMetadata': {'RequestId': '20153835-a1cb-4d8b-b31b-fc5194cf9a11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Apr 2020 23:21:58 GMT',
   'x-amzn-requestid': '20153835-a1cb-4d8b-b31b-fc5194cf9a11',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Giving Transcriptions to Amazon Polly

Amazon Polly is queried for each individual word to allow for easier control of timing and pitch.

In [21]:
# Adjustable Variable:

# Short words tend to be transcribed too short. So this manually extends them. 
extend_word_length_factor = 300 # (percent of total word duration)
word_under_x_ms_long = 750

add_pause_for_gaps_greater_than = 100 # ms

In [6]:
def query_polly(polly_client, word, length, prefix, output_folder):
    
    ssml = """<speak><prosody amazon:max-duration="{max_len}ms">{word}</prosody></speak>""".format(max_len=str(length), word=word)          
    response = polly_client.start_speech_synthesis_task(VoiceId='Joey',
                OutputS3BucketName='sagemaker-us-east-2-075178354542',
                OutputS3KeyPrefix='polly-output/' + prefix,
                OutputFormat='mp3', 
                TextType = 'ssml',
                Text = ssml)


In [16]:
# Patching the batches back together, generate transcription list from all the batches. 
transcribe_output_folder = "transcribe-output/"
offset = 0 # Takes into account that batches are sequential.
transcription_list = list()
for file in sorted(os.listdir(transcribe_output_folder)):
    transcription_batch = json.load(open(transcribe_output_folder + file, "r", encoding="utf-8"))
    for map_item in transcription_batch["results"]["items"]:
        transcribe_object = processing_util.TranscriptionItem(map_item, offset)
        # Skip punctuation
        if transcribe_object.is_word():
            transcription_list.append(transcribe_object)
    offset += 30000

In [22]:
polly_client = boto3.client('polly')

transcribe_output_folder = "transcribe-output/"
polly_output_folder = "polly-output/"
audio_util.clear_folder(polly_output_folder)
silence_dict = {"length" : 0}
index = 0
expected_start_time = 0

for i, transcribe_object in enumerate(transcription_list):
    
    # Create pause if there is a larger than 1 second gap between words.
    if expected_start_time + add_pause_for_gaps_greater_than < transcribe_object.start_time:
        silence_dict["length"] = transcribe_object.start_time - expected_start_time
        with open(polly_output_folder + str(index).zfill(5) + ".json", 'w') as outfile:
            json.dump(silence_dict, outfile)
        expected_start_time = transcribe_object.start_time
        index += 1
    
    if transcribe_object.duration() < word_under_x_ms_long:
        transcribe_object.end_time += extend_word_length_factor
        
    response = query_polly(polly_client, transcribe_object.content, transcribe_object.duration(), str(index).zfill(5), polly_output_folder)
    expected_start_time = transcribe_object.end_time
    print("Polly Queried " + str(index) + " for: " + transcribe_object.content, "| Word Duration(ms): " + str(transcribe_object.duration()), "| Transcription Confidence: " + str(transcribe_object.confidence))  
    print(expected_start_time)
    index += 1

Polly Queried 0 for: Oh | Word Duration(ms): 816.0 | Transcription Confidence: 0.871
856.0
Polly Queried 2 for: black | Word Duration(ms): 852.0 | Transcription Confidence: 1.0
10962.0
Polly Queried 3 for: leather | Word Duration(ms): 744.0 | Transcription Confidence: 1.0
11324.0
Polly Queried 4 for: blood | Word Duration(ms): 780.0 | Transcription Confidence: 0.9456
11740.0
Polly Queried 5 for: No | Word Duration(ms): 600.0 | Transcription Confidence: 0.8913
11960.0
Polly Queried 6 for: seafood | Word Duration(ms): 780.0 | Transcription Confidence: 0.9966
12390.0
Polly Queried 8 for: buckles | Word Duration(ms): 860.0 | Transcription Confidence: 0.9944
13950.0
Polly Queried 9 for: on | Word Duration(ms): 504.0 | Transcription Confidence: 0.9998
14154.0
Polly Queried 10 for: a | Word Duration(ms): 432.0 | Transcription Confidence: 0.9204
14252.0
Polly Queried 11 for: jacket | Word Duration(ms): 828.0 | Transcription Confidence: 0.999
14758.0
Polly Queried 12 for: It's | Word Duration(m

Polly Queried 100 for: wanna | Word Duration(ms): 708.0 | Transcription Confidence: 0.6373
47358.0
Polly Queried 101 for: dance | Word Duration(ms): 876.0 | Transcription Confidence: 1.0
47876.0
Polly Queried 102 for: with | Word Duration(ms): 480.0 | Transcription Confidence: 1.0
47960.0
Polly Queried 103 for: me | Word Duration(ms): 840.0 | Transcription Confidence: 1.0
48470.0
Polly Queried 104 for: No | Word Duration(ms): 860.0 | Transcription Confidence: 0.6201
49030.0
Polly Queried 105 for: I | Word Duration(ms): 480.0 | Transcription Confidence: 0.9999
49420.0
Polly Queried 106 for: could | Word Duration(ms): 564.0 | Transcription Confidence: 0.9827
49654.0
Polly Queried 107 for: guess | Word Duration(ms): 576.0 | Transcription Confidence: 0.7737
49886.0
Polly Queried 108 for: I'm | Word Duration(ms): 468.0 | Transcription Confidence: 0.7271
50008.0
Polly Queried 109 for: Michael | Word Duration(ms): 636.0 | Transcription Confidence: 0.9955
50316.0
Polly Queried 110 for: J | Wor

Polly Queried 196 for: so | Word Duration(ms): 444.0 | Transcription Confidence: 0.5213
86644.0
Polly Queried 197 for: mean | Word Duration(ms): 816.0 | Transcription Confidence: 0.9885
87136.0
Polly Queried 198 for: being | Word Duration(ms): 636.0 | Transcription Confidence: 0.4644
87656.0
Polly Queried 199 for: mistaken | Word Duration(ms): 870.0 | Transcription Confidence: 0.39
88170.0
Polly Queried 200 for: for | Word Duration(ms): 468.0 | Transcription Confidence: 0.5765
88518.0
Polly Queried 201 for: other | Word Duration(ms): 744.0 | Transcription Confidence: 0.9745
88944.0
Polly Queried 203 for: people | Word Duration(ms): 804.0 | Transcription Confidence: 1.0
90374.0
Polly Queried 204 for: would | Word Duration(ms): 480.0 | Transcription Confidence: 0.4719
90480.0
Polly Queried 205 for: at | Word Duration(ms): 492.0 | Transcription Confidence: 0.5898
90642.0
Polly Queried 206 for: a | Word Duration(ms): 408.0 | Transcription Confidence: 0.5267
90718.0
Polly Queried 207 for: l

Polly Queried 287 for: it | Word Duration(ms): 636.0 | Transcription Confidence: 0.9471
117786.0
Polly Queried 288 for: Go | Word Duration(ms): 552.0 | Transcription Confidence: 0.9764
117982.0
Polly Queried 289 for: right | Word Duration(ms): 636.0 | Transcription Confidence: 0.9978
118276.0
Polly Queried 290 for: foot | Word Duration(ms): 792.0 | Transcription Confidence: 0.6577
118722.0
Polly Queried 292 for: that | Word Duration(ms): 732.0 | Transcription Confidence: 0.9907
121252.0
Polly Queried 293 for: foot | Word Duration(ms): 804.0 | Transcription Confidence: 0.9761
121694.0
Polly Queried 295 for: right | Word Duration(ms): 820.0 | Transcription Confidence: 1.0
122700.0
Polly Queried 296 for: foot | Word Duration(ms): 744.0 | Transcription Confidence: 0.9922
123154.0
Polly Queried 298 for: Basically | Word Duration(ms): 970.0 | Transcription Confidence: 0.9993
124380.0
Polly Queried 299 for: I'm | Word Duration(ms): 396.0 | Transcription Confidence: 0.9788
124476.0
Polly Queri

Polly Queried 380 for: with | Word Duration(ms): 708.0 | Transcription Confidence: 0.9998
150348.0
Polly Queried 381 for: do | Word Duration(ms): 528.0 | Transcription Confidence: 0.7883
150528.0
Polly Queried 382 for: this | Word Duration(ms): 444.0 | Transcription Confidence: 0.5132
150634.0
Polly Queried 383 for: shit | Word Duration(ms): 588.0 | Transcription Confidence: 0.989
150898.0
Polly Queried 384 for: ourselves | Word Duration(ms): 860.0 | Transcription Confidence: 0.7018
151410.0
Polly Queried 385 for: But | Word Duration(ms): 588.0 | Transcription Confidence: 0.9251
151708.0
Polly Queried 386 for: if | Word Duration(ms): 612.0 | Transcription Confidence: 1.0
152032.0
Polly Queried 387 for: I'm | Word Duration(ms): 540.0 | Transcription Confidence: 0.796
152220.0
Polly Queried 388 for: over | Word Duration(ms): 564.0 | Transcription Confidence: 0.8248
152444.0
Polly Queried 389 for: shake | Word Duration(ms): 672.0 | Transcription Confidence: 0.8968
152772.0
Polly Queried 3

Polly Queried 470 for: we're | Word Duration(ms): 468.0 | Transcription Confidence: 0.2179
184468.0
Polly Queried 471 for: about | Word Duration(ms): 576.0 | Transcription Confidence: 0.7247
184716.0
Polly Queried 472 for: to | Word Duration(ms): 372.0 | Transcription Confidence: 0.423
184752.0
Polly Queried 473 for: start | Word Duration(ms): 648.0 | Transcription Confidence: 0.3038
185098.0
Polly Queried 474 for: I | Word Duration(ms): 804.0 | Transcription Confidence: 0.4199
185544.0
Polly Queried 475 for: can | Word Duration(ms): 708.0 | Transcription Confidence: 0.3018
185878.0
Polly Queried 476 for: let | Word Duration(ms): 480.0 | Transcription Confidence: 0.3573
185990.0
Polly Queried 477 for: this | Word Duration(ms): 516.0 | Transcription Confidence: 0.9993
186176.0
Polly Queried 478 for: one | Word Duration(ms): 432.0 | Transcription Confidence: 0.8625
186272.0
Polly Queried 479 for: slide | Word Duration(ms): 852.0 | Transcription Confidence: 0.3347
186802.0
Polly Queried 4

Polly Queried 562 for: saying | Word Duration(ms): 876.0 | Transcription Confidence: 1.0
219216.0
Polly Queried 563 for: either | Word Duration(ms): 600.0 | Transcription Confidence: 0.9287
219420.0
Polly Queried 564 for: way | Word Duration(ms): 4740.0 | Transcription Confidence: 0.9983
223820.0


### Processing the Output from Amazon Polly

In [23]:
import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "polly-output/"

for object_summary in my_bucket.objects.filter(Prefix=prefix):
    file_name = object_summary.key.split('/')[-1]
    my_bucket.download_file(prefix+ file_name, prefix + file_name)
    
print("Files moved from s3 to repo.")

Files moved from s3 to repo.


Mixing Audio:

https://stackoverflow.com/questions/7629873/how-do-i-mix-audio-files-using-python

Pitch Modulation:

https://stackoverflow.com/questions/38923438/does-pydub-support-pitch-modulation

Create Video:

https://helpdeskgeek.com/windows-10/how-to-record-your-screen-on-windows-10/

Music Video:

https://www.oneimagevideo.com/